In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

import sys
sys.path.append('./src/misc')
import tools
import sql_tools

In [4]:
path_to_data = "./src/sql/"
data = sql_tools.read_database(path_to_data+"trailing_database.db", "5_game_trailing")
data = data[data['O/U_line']!='']
data['O/U_result'] = data.apply(lambda row: tools.OU(row['O/U_line'],float(row['total'])),axis=1)

In [5]:
X,y = tools.model_preprocessing(data,("2016-01-10","2023-12-12"))

## Random Forest

In [4]:
n = 15
n_estimators = 50
n_trials = 10
OU_results = list(y[-n:])

acc_vals = []

print(f"Backtesting for {n} games with {n_estimators} estimators with {n_trials} trials.\n")
for trial in tqdm(range(0,n_trials)):
    pred = []
    for i in range(0,n):
        rf_model = RandomForestClassifier(n_estimators=n_estimators)
        
        X_train = X[n+1-i:]
        y_train = y[n+1-i:]
        
        X_test = X[n-i]
        y_test = y[n-i]
        
        rf_model.fit(X_train,y_train)
        yhat = rf_model.predict(X_test.reshape(1, -1))
        
        pred.append(yhat[0])

    acc = tools.score_results(OU_results,pred)
    acc_vals.append(acc)

  0%|          | 0/20 [00:00<?, ?it/s]

Trial 0


  5%|▌         | 1/20 [00:16<05:22, 16.98s/it]

predicted:  [1, 1, 1, 1, 1, 1, 0, 0, 0, 1]
true:       [1, 0, 0, 1, 0, 1, 1, 2, 0, 1]
accuracy:   50.0
Trial 1


 10%|█         | 2/20 [00:33<05:05, 16.98s/it]

...

predicted:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1]
true:       [1, 0, 0, 1, 0, 1, 1, 2, 0, 1]
accuracy:   50.0
Trial 18


 95%|█████████▌| 19/20 [05:21<00:16, 16.86s/it]

predicted:  [1, 0, 1, 1, 0, 1, 0, 0, 0, 1]
true:       [1, 0, 0, 1, 0, 1, 1, 2, 0, 1]
accuracy:   70.0
Trial 19


100%|██████████| 20/20 [05:38<00:00, 16.90s/it]

predicted:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1]
true:       [1, 0, 0, 1, 0, 1, 1, 2, 0, 1]
accuracy:   50.0


In [11]:
overall_acc = sum(acc_vals)/len(acc_vals)
print(f"Random Forest Classifier achieved {overall_acc*100}% overall accruacy for the past {n} games. Accuracy is calculated from {20} trials.")

0.5399999999999999

## NN Model

In [191]:
import tensorflow as tf
from tensorflow import keras

#earling stopping
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model = tf.keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = X[0].shape))
model.add(tf.keras.layers.Dense(128, activation="relu6"))
model.add(keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(3, activation="softmax"))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [197]:
X,y = tools.model_preprocessing(data,("2017-01-10","2023-12-12"))

callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model.fit(X[301:], 
          y[301:], 
          epochs=100, 
          validation_split=0.1, 
          batch_size=100,
          verbose=0,
          callbacks=[callback])

print(f"Backtesting for {n} games with {n_trials} trials.\n")

n = 300
OU_results = list(y[-n:])
pred = []
acc_vals = []

for i in tqdm(range(0,n)):
  #X_train = X[n+1-i:]
  #y_train = y[n+1-i:]
  
  if i%10:
      model.fit(X[n+1-i:], 
          y[n+1-i:], 
          epochs=100, 
          validation_split=0.1, 
          batch_size=100,
          callbacks=[callback])

  X_test = X[n-i]
  y_test = y[n-i]
  
  prediction_output = model.predict(X_test.reshape(1,-1),verbose=0)
  pred.append(tools.prediction_classifier(prediction_output)[0])

acc = tools.score_results(OU_results,pred)
acc_vals.append(acc)
  
overall_acc = sum(acc_vals)/len(acc_vals)
print(f"\nNeural network achieved {overall_acc*100}% overall accruacy for the past {n} games. Accuracy is calculated from {n_trials} trials.")

Backtesting for 299 games with 1 trials.



  0%|          | 0/300 [00:00<?, ?it/s]

Epoch 1/100
65/65 [==============================] - 0s 7ms/step - loss: 0.3896 - accuracy: 0.8589 - val_loss: 0.4069 - val_accuracy: 0.8707
Epoch 2/100
65/65 [==============================] - 0s 6ms/step - loss: 0.4078 - accuracy: 0.8392 - val_loss: 0.4064 - val_accuracy: 0.8595
Epoch 3/100
65/65 [==============================] - 0s 6ms/step - loss: 0.3893 - accuracy: 0.8589 - val_loss: 0.4060 - val_accuracy: 0.8762
Epoch 4/100
65/65 [==============================] - 0s 7ms/step - loss: 0.3941 - accuracy: 0.8528 - val_loss: 0.4023 - val_accuracy: 0.8748
Epoch 5/100
65/65 [==============================] - 1s 8ms/step - loss: 0.3795 - accuracy: 0.8657 - val_loss: 0.4051 - val_accuracy: 0.8720
Epoch 6/100
65/65 [==============================] - 0s 7ms/step - loss: 0.3879 - accuracy: 0.8587 - val_loss: 0.4017 - val_accuracy: 0.8665
Epoch 7/100
65/65 [==============================] - 0s 7ms/step - loss: 0.3845 - accuracy: 0.8593 - val_loss: 0.4130 - val_accuracy: 0.8526
Epoch 8/100
6

  1%|          | 2/300 [00:15<37:15,  7.50s/it]

Epoch 1/100
65/65 [==============================] - 1s 9ms/step - loss: 0.3573 - accuracy: 0.8691 - val_loss: 0.3722 - val_accuracy: 0.8637
Epoch 2/100
65/65 [==============================] - 0s 7ms/step - loss: 0.3575 - accuracy: 0.8708 - val_loss: 0.3736 - val_accuracy: 0.8693
Epoch 3/100
65/65 [==============================] - 0s 7ms/step - loss: 0.3562 - accuracy: 0.8658 - val_loss: 0.3716 - val_accuracy: 0.8762
Epoch 4/100
65/65 [==============================] - 0s 7ms/step - loss: 0.3517 - accuracy: 0.8736 - val_loss: 0.3693 - val_accuracy: 0.8748
Epoch 5/100
65/65 [==============================] - 0s 7ms/step - loss: 0.3532 - accuracy: 0.8675 - val_loss: 0.4070 - val_accuracy: 0.8414
Epoch 6/100
65/65 [==============================] - 0s 7ms/step - loss: 0.3484 - accuracy: 0.8743 - val_loss: 0.3675 - val_accuracy: 0.8665
Epoch 7/100
65/65 [==============================] - 0s 7ms/step - loss: 0.3550 - accuracy: 0.8689 - val_loss: 0.4113 - val_accuracy: 0.8401
Epoch 8/100
6

...

Epoch 1/100
66/66 [==============================] - 1s 11ms/step - loss: 0.2723 - accuracy: 0.8970 - val_loss: 0.3861 - val_accuracy: 0.8368
Epoch 2/100
66/66 [==============================] - 0s 8ms/step - loss: 0.2894 - accuracy: 0.8867 - val_loss: 0.4405 - val_accuracy: 0.8216
Epoch 3/100
66/66 [==============================] - 1s 8ms/step - loss: 0.2783 - accuracy: 0.8913 - val_loss: 0.3603 - val_accuracy: 0.8575
Epoch 4/100
66/66 [==============================] - 1s 8ms/step - loss: 0.2758 - accuracy: 0.8939 - val_loss: 0.4095 - val_accuracy: 0.8313
Epoch 5/100
66/66 [==============================] - 0s 8ms/step - loss: 0.2827 - accuracy: 0.8896 - val_loss: 0.3329 - val_accuracy: 0.8769
Epoch 6/100
66/66 [==============================] - 1s 8ms/step - loss: 0.2667 - accuracy: 0.8990 - val_loss: 0.3666 - val_accuracy: 0.8506
Epoch 7/100
66/66 [==============================] - 1s 8ms/step - loss: 0.2676 - accuracy: 0.9010 - val_loss: 0.3314 - val_accuracy: 0.8769
Epoch 8/100


 14%|█▍        | 43/300 [04:01<21:28,  5.01s/it]

Epoch 1/100
66/66 [==============================] - 1s 12ms/step - loss: 0.2663 - accuracy: 0.8998 - val_loss: 0.3354 - val_accuracy: 0.8714
Epoch 2/100
66/66 [==============================] - 1s 9ms/step - loss: 0.2555 - accuracy: 0.9053 - val_loss: 0.3476 - val_accuracy: 0.8672
Epoch 3/100
66/66 [==============================] - 1s 8ms/step - loss: 0.2750 - accuracy: 0.8952 - val_loss: 0.4991 - val_accuracy: 0.7967
Epoch 4/100
66/66 [==============================] - 1s 9ms/step - loss: 0.2726 - accuracy: 0.8972 - val_loss: 0.3439 - val_accuracy: 0.8728
Epoch 5/100
66/66 [==============================] - 1s 8ms/step - loss: 0.2750 - accuracy: 0.8932 - val_loss: 0.4827 - val_accuracy: 0.8064
Epoch 6/100
66/66 [==============================] - 1s 8ms/step - loss: 0.2842 - accuracy: 0.8892 - val_loss: 0.3599 - val_accuracy: 0.8631
Epoch 7/100
66/66 [==============================] - 1s 8ms/step - loss: 0.2683 - accuracy: 0.8989 - val_loss: 0.3352 - val_accuracy: 0.8741


 15%|█▍        | 44/300 [04:05<20:22,  4.77s/it]

Epoch 1/100
66/66 [==============================] - 1s 8ms/step - loss: 0.2767 - accuracy: 0.8975 - val_loss: 0.3416 - val_accuracy: 0.8702
Epoch 2/100
53/66 [=======================>......] - ETA: 0s - loss: 0.2702 - accuracy: 0.8972